# INTRODUCTION TO TENSORFLOW 2.0

- There are tons of new features for Tensorflow 2.0, here we will introduce 4 of them:
  - 1. FEATURE #1: EAGER EXECUTION BY DEFAULT
  - 2. FEATURE #2: KERAS IS THE DEFAULT API (EASIER THAN EVER TO BUILD, TRAIN AND DEBUG MODELS) 
  - 3. FEATURE #3: TENSORBOARD
  - 4. FEATURE #4: DISTRIBUTED STRATEGY 



# 1. FEATURE #1: EAGER EXECUTION BY DEFAULT

- Tensorflow now enables eager execution by default!
- Tensorflow now has eager execution by default which means you can evaluate operations immediately 
- This will make your life 10 times easier when you build and debug your AI model. 
- Eager execution means that you can now interact with TF 2.0 line by line in google colab or jupyter notebook without the need to define a graph and run sessions and all the mess we had with tensorflow 1.0.


# *1.A ADD TWO VARIABLES IN TENSORFLOW 1.0*

In [0]:
!pip install tensorflow-gpu==1.13.01

In [0]:
import tensorflow as tf

# First we have a "construction phase" where we build a graph
x = tf.Variable(3)
y = tf.Variable(5)

# Tensorflow created a graph but did not execute the graph yet
# A session is needed to run the graph 
z = tf.add(x,y)

print("The sum of x and y is:", z) # we did not get the expected answer, i.e: 8


In [0]:

# "Execution phase" where we run a session 
# this makes it super difficult to debug and develop models
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer()) #initialize all variables
  z = sess.run(z) #run the session
print("The sum of x and y is:", z) # we now get the expected answer, i.e: 8


# *1.B LET'S ADD TWO VARIABLES IN TENSORFLOW 2.0*

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

In [0]:
# TEnsorflow 2.0 still works with graphs but enable eager execution by default
# Let's add the same variables together
import tensorflow as tf
x = tf.Variable(3)
y = tf.Variable(5)
z = tf.add(x,y) # immediate answer! 
print("The sum of x and y is:", z) # we get the answer immediately!


# 2. FEATURE #2: KERAS IS THE DEFAULT API (EASIER THAN EVER TO TRAIN AND DEBUG) 


- The second important feature in TF 2.0 is the use of keras as the high level API by default 
- Keras is unbelievably easy to work with! 
- Keras syntax is very pythonic and for those of you who have worked with Python before will know that python language is super easy to learn!
- Let's build a mini artificial neural network that can classify fashion images using keras API. 

# *LET'S TRAIN A MINI NETWORK FOR IMAGE CLASSIFICATION USING TF 2.0 AND KERAS API*

In [0]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs=5)


# 3. FEATURE #3: TENSORBOARD

- Tensorboard enable us to track the network progress such as accuracy and loss throughout various epochs along with the graph showing various layers of the network which is pretty incredible!
- In addition, tensorboard provides a built-in performance dashboard that can be used to track device placement and help us minimize bottlenecks during model execution and training. 
Let me show you how to launch tensorboard and how it works: 


# *3.1 EXAMPLE #1*

In [0]:
!pip install tensorflow-gpu==1.13.01 # Make Sure the Runtime Type is set to "Python 3" and "GPU"


!pip install -q tf-nightly-2.0-preview
# !pip install -q tf-nightly-gpu-2.0-preview
%load_ext tensorboard


In [0]:
# !pip install tensorflow-gpu==2.0.0.alpha0
import tensorflow as tf
import datetime

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist


(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(train_images, train_labels, epochs=5, callbacks = [tensorboard_callback])


In [0]:
%tensorboard --logdir logs/fit

# *3.2 EXAMPLE #2*

In [0]:


fashion_mnist = tf.keras.datasets.fashion_mnist


(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


# Reshape training data to be = (60000, 28, 28, 1) instead of (60000, 28,28)
train_images = train_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)


from tensorflow.keras import datasets, layers, models

model = models.Sequential()

model.add(layers.Conv2D(6, (5,5), activation = 'relu', input_shape = (28,28,1)))
model.add(layers.AveragePooling2D())


model.add(layers.Conv2D(16, (5,5), activation = 'relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(120, activation = 'relu'))

model.add(layers.Dense(84, activation = 'relu'))

model.add(layers.Dense(10, activation = 'softmax'))
model.summary()



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(train_images, train_labels, epochs=5, callbacks = [tensorboard_callback])


In [0]:
%tensorboard --logdir logs/fit

# 4. FEATURE #4: DISTRIBUTED STRATEGY 

- Tensorflow enables distributed strategy which allows developers to develop the model once and then decide how do they want to run it, over multiple GPUs or TPUs. 
- This will dramatically improve the computational efficiency with just two additional lines of code


In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

import tensorflow as tf
import datetime

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])


  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

     |████████████████████████████████| 332.1MB 56kB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
     |████████████████████████████████| 419kB 49.7MB/s 
4423680/4422102 [==============================] - 0s 0us/step


In [0]:
model.fit(train_images, train_labels, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 3.4091 - accuracy: 0.6775
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.8112 - accuracy: 0.6977
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.6915 - accuracy: 0.7272
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.6290 - accuracy: 0.7591
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5800 - accuracy: 0.7887


# EXCELLENT JOB! NOW YOU'RE FAMILIAR WITH TF 2.0!